[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/axr/blob/master/axr/00_intro.ipynb)

# Introducción

El aprendizaje por refuerzo explora una aproximación computacional al aprendizaje por interacción. De la misma manera que cuando aprendemos a conducir estamos atentos a cómo reacciona nuestro entorno a nuestras acciones y buscamos maneras de influenciarlo a través de nuestro comportamiento, el aprendizaje por refuerzo estudia cómo agentes computacionales pueden desarrollar comportamientos inteligentes a través de este tipo de interacción.

## Aprendizaje por refuerzo


El aprendizaje por refuerzo, de ahora en adelante **axr**, consiste en aprender qué hacer (cómo relacionar situaciones y acciones) con el objetivo de maximizar una recompensa numérica. En ningún momento especificamos qué acciones debe tomar un agente, sino que le dejamos descubrir cuales son las que le darán una mayor recompensa. En la mayoría de situaciones, una acción no solo afectará a la recompensa inmediata, si no que tendrá un efecto en todas las situaciones futuras. Estas dos propiedades, búsqueda por prueba y error y futuras recompensas, son las más importantes del axr.

Para formalizar el problema del axr utilizamos ideas del campo de la teoría de sistemas dinámicos, en concreto el control óptimo de procesos de Markov incompletos. La idea básica consiste en aprender los aspectos más importantes sobre el problema real al que nuestro agente se enfrenta a través de la interacción con el entrono para conseguir su objetivo. El agente tiene que ser capaz de precibir su entorno y de llevar a cabo acciones que afecten a su estado. También necesita uno o varios objetivos relacionados con el estado del entorno. Un proceso de decisión de Markov incluye estos tres aspectos: percepción, acción y objetivo. Cualquier método que sea capaz de resolver este tipo de problemas se considera como un método de axr.

El axr está considerado como un paradigma del *machine learning* diferente al aprendizaje supervisado y no supervisado. Se diferencia del aprendizaje supervisado en que no siempre será posible obtener ejemplos del comportamiento deseado para nuestro agente en cualquier tipo de situación en la que se pueda encontrar, por lo que deberá ser capaz de aprender de su propia experiencia. Por otro lado, se diferencia del aprendizaje no supervisado ya que éste no es capaz por si mismo de resolver el problema de maximización de la recompensa.

El principal problema al que nos enfrentamos en el axr es el balance entre **exploración** y **explotación**. Para obtener la máxima recompensa, un agente podría escoger aquellas acciones que ya conoce que y que le direon buenos resultados. Sin embargo, el hecho de explorar nuevas acciones podría, eventualmente, dar mucho mejor resultado. Así pues, nuestro agente tiene que ser capaz de explotar su conocimiento para obtener recompensa pero también de explorar para descubrir mejores acciones. El problema es que ninguna de las dos aproximaciones puede llevarse a cabo de manera independiente para resolver un problema. Un agente debe probar muchas acciones y, poco a poco, favorecer aquellas que parezcan ser mejores. Este problema sigue sin estar resuelto.

Otra propiedad importante que diferencia al axr de otro métodos es su capacidad de considerar todo el dominio del problema de un agente interactuando con su entrono, y no pequeñas partes o sub-tareas que puedan resolverse de manera independiente para llevar al objetivo final.

## Elementos del axr


Clase del Tablero (ConnectFourBoard)

In [3]:
import numpy as np

class ConnectFourBoard():
    def __init__(self):
        self.state = np.zeros((6, 7))  # Tablero de 6 filas x 7 columnas
        self.current_player = 1

    def valid_moves(self):
        """Devuelve las columnas disponibles (0-6) donde se puede colocar ficha"""
        return [col for col in range(7) if self.state[0, col] == 0]

    def update(self, symbol, col):
        """Coloca una ficha en la columna especificada"""
        if col not in self.valid_moves():
            raise ValueError("Movimiento ilegal: columna llena o inválida!")

        # Encontrar la primera fila vacía en la columna
        for row in range(5, -1, -1):
            if self.state[row, col] == 0:
                self.state[row, col] = symbol
                break

    def is_game_over(self):
        # Comprobar 4 en línea horizontal
        for row in range(6):
            for col in range(4):
                if abs(sum(self.state[row, col:col+4])) == 4:
                    return self.state[row, col]

        # Comprobar 4 en línea vertical
        for col in range(7):
            for row in range(3):
                if abs(sum(self.state[row:row+4, col])) == 4:
                    return self.state[row, col]

        # Comprobar diagonales (pendiente positiva)
        for row in range(3):
            for col in range(4):
                if abs(sum(self.state[row+i, col+i] for i in range(4))) == 4:
                    return self.state[row, col]

        # Comprobar diagonales (pendiente negativa)
        for row in range(3, 6):
            for col in range(4):
                if abs(sum(self.state[row-i, col+i] for i in range(4))) == 4:
                    return self.state[row, col]

        # Empate (tablero lleno)
        if len(self.valid_moves()) == 0:
            return 0

        # El juego continúa
        return None

    def reset(self):
        self.state = np.zeros((6, 7))
        self.current_player = 1

    def print_board(self):
        """Muestra el tablero de forma legible"""
        symbols = {0: '·', 1: 'X', -1: 'O'}
        print("\n  0 1 2 3 4 5 6")
        for row in self.state:
            print("  " + " ".join(symbols[cell] for cell in row))
        print()

Clase del Juego (ConnectFourGame)

In [4]:
from tqdm import tqdm

class ConnectFourGame():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = ConnectFourBoard()

    def selfplay(self, rounds=100):
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()

            game_over = False
            while not game_over:
                for player in self.players:
                    col = player.move(self.board)
                    self.board.update(player.symbol, col)

                    for p in self.players:
                        if hasattr(p, 'update'):
                            p.update(self.board)

                    if self.board.is_game_over() is not None:
                        game_over = True
                        break

            self.reward()

            winner = self.board.is_game_over()
            for ix, player in enumerate(self.players):
                if winner == player.symbol:
                    wins[ix] += 1

        return wins

    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0:  # empate
            for player in self.players:
                player.reward(0.5)
        else:
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

Clase del Agente (Agent)

In [5]:
import numpy as np

class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {}  # Diccionario estado -> valor
        self.alpha = alpha        # Tasa de aprendizaje
        self.positions = []       # Historial de posiciones en la partida
        self.prob_exp = prob_exp  # Probabilidad de exploración
        self.symbol = None        # Se establecerá al iniciar el juego

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()

        # Exploración: movimiento aleatorio
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            return np.random.choice(valid_moves)

        # Explotación: elegir mejor movimiento según value_function
        max_value = -float('inf')
        best_col = valid_moves[0]  # Por defecto, primera columna válida

        for col in valid_moves:
            # Simular el movimiento
            next_board = board.state.copy()
            for row in range(5, -1, -1):
                if next_board[row, col] == 0:
                    next_board[row, col] = self.symbol
                    break

            # Obtener valor del estado resultante
            state_key = str(next_board.reshape(6*7))
            value = self.value_function.get(state_key, 0)

            if value > max_value:
                max_value = value
                best_col = col

        return best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(6*7)))

    def reward(self, reward):
        # Actualizar value_function basado en recompensa
        for state in reversed(self.positions):
            if state not in self.value_function:
                self.value_function[state] = 0
            self.value_function[state] += self.alpha * (reward - self.value_function[state])
            reward = self.value_function[state]

Entrenamiento de los Agentes

In [6]:
# Crear agentes
agent1 = Agent(prob_exp=0.5)  # Mayor exploración inicial
agent2 = Agent(prob_exp=0.3)  # Menos exploración

# Crear juego
game = ConnectFourGame(agent1, agent2)

# Entrenamiento (puedes reducir las rondas para probar)
results = game.selfplay(10000)  # Usar menos rondas para prueba inicial
print("Resultados del entrenamiento:", results)

100%|██████████| 10000/10000 [05:25<00:00, 30.69it/s]

Resultados del entrenamiento: [5915, 4027]


Visualización de la Función de Valor

In [7]:
import pandas as pd

# Ordenar estados por valor
sorted_values = sorted(agent1.value_function.items(), key=lambda x: x[1], reverse=True)

# Crear DataFrame para visualización
value_df = pd.DataFrame({
    'Estado': [x[0] for x in sorted_values[:10]],  # Mostrar solo los 10 mejores
    'Valor': [x[1] for x in sorted_values[:10]]
})

value_df

,Estado,Valor
0,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1.0
1,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1.0
2,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1.0
3,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1.0
4,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1.0
5,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1.0
6,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1.0
7,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1.0
8,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1.0
9,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,1.0


Guardar el Agente Entrenado

In [8]:
import pickle

with open('connect4_agent.pickle', 'wb') as f:
    pickle.dump(agent1.value_function, f)

Este ejemplo sirve para ilustrar algunas de las propiedades clave del axr. En primer lugar, aprender a través de la interacción con el entorno (en este caso el otro agente). En segundo lugar, tenemos un objetivo claro y el comportamiento correcto del agente requiere de planificación y predicción que tenga en cuenta los efectos futuros de sus acciones.

##  Resumen

El aprendizaje por refuerzo es una aproximación computacional a la comprensión y automatización del aprendizaje por objetivos y toma de decisiones. En esta aproximación, una agente aprende a través de la interacción directa con su entorno sin necesidad de supervisión explícita. Utiliza procesos de decisión de Markov para definir la interacción entre el agente y su entorno en términos de estados, acciones y recompensas. Los conceptos de valor y función de valor son la clave de muchos métodos de axr ya que representan una manera eficiente de búsqueda en el espacio de políticas.